<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb monte_carlo_v2.do.txt --encoding=utf-8 --ipynb_admon=hrule --ipynb_disable_mpl_inline --ipynb_cite=latex-plain -->

<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb monte_carlo.do.txt --encoding=utf-8 --ipynb_admon=hrule --ipynb_disable_mpl_inline --ipynb_cite=latex-plain

<!-- Monte Carlo sensitivity analysis (Sobol indices) -->

**Leif Rune Hellevik**

Date: **January 22, 2026**

# Setup

In [1]:
# --- cell: install_chaospy ---
# @title Install chaospy (Colab-friendly)

try:
    import chaospy as cp
    import numpoly
    import numpy as np
    print("chaospy er allerede installert.")
except ImportError:
    # Installer chaospy fra PyPI. Dette drar inn numpoly automatisk.
    %pip install chaospy==4.3.21 --no-cache-dir
    import chaospy as cp
    import numpoly
    import numpy as np

print("numpy  :", np.__version__)
print("numpoly:", numpoly.__version__)
print("chaospy:", cp.__version__)

chaospy er allerede installert.
numpy  : 2.2.6
numpoly: 1.3.6
chaospy: 4.3.20


In [2]:
# --- cell: repo_setup ---
# @title Repo sync and environment setup

import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules
REMOTE = "https://github.com/lrhgit/uqsa2025.git"
REPO_PATH_COLAB = Path("/content/uqsa2025")

if IN_COLAB:
    if not REPO_PATH_COLAB.exists():
        print("Cloning repository...")
        subprocess.run(
            ["git", "clone", REMOTE, str(REPO_PATH_COLAB)],
            check=True
        )
    else:
        print("Updating existing repository...")
        subprocess.run(
            ["git", "-C", str(REPO_PATH_COLAB), "pull"],
            check=True
        )
    os.chdir(REPO_PATH_COLAB)

# --- Find repo root (works locally + in Colab) ---
cwd = Path.cwd().resolve()
repo_root = next(
    (p for p in [cwd] + list(cwd.parents) if (p / ".git").exists()),
    cwd
)

PY_SRC = repo_root / "python_source"
if PY_SRC.exists() and str(PY_SRC) not in sys.path:
    sys.path.insert(0, str(PY_SRC))

print("CWD:", Path.cwd())
print("repo_root:", repo_root)
print("python_source exists:", PY_SRC.exists())
print("python_source in sys.path:", str(PY_SRC) in sys.path)

CWD: /Users/leifh/git/uqsa2025
repo_root: /Users/leifh/git/uqsa2025
python_source exists: True
python_source in sys.path: True


In [3]:
# --- cell: layout_and_numpy_patch ---
# @title Layout fix, imports, and NumPy compatibility patch

import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML

HTML("""
<style>
div.cell.code_cell, div.output {
    max-width: 100% !important;
}
</style>
""")

import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
import numpoly
import pandas as pd


# Pretty-print helpers (used across notebooks)
from pretty_printing import section_title, pretty_table, pretty_print_sobol_mc


# --- NumPy reshape compatibility patch for numpoly ---
_old_reshape = np.reshape

def _reshape_compat(a, *args, **kwargs):
    newshape = None
    if "newshape" in kwargs:
        newshape = kwargs.pop("newshape")
    if "shape" in kwargs and newshape is None:
        newshape = kwargs.pop("shape")
    if newshape is not None:
        return _old_reshape(a, newshape, *args, **kwargs)
    return _old_reshape(a, *args, **kwargs)

np.reshape = _reshape_compat
print("✓ numpy.reshape patched for numpoly compatibility")

✓ numpy.reshape patched for numpoly compatibility


# Monte Carlo: skeleton

<!-- (fill in later) -->

In [4]:
# --- cell: linear_model ---

import numpy as np

def linear_model(w, z):
    """
    Linear model used across the entire notebook:
        Y = sum_i (w_i * z_i)

    Parameters
    ----------
    w : array-like, shape (Nrv,)
        Weights for each random variable.

    z : array-like, shape (Ns, Nrv)
        Sample matrix where rows are samples and columns correspond to random variables.

    Returns
    -------
    Y : array-like, shape (Ns,)
        Model outputs for each sample.
    """
    return np.sum(w * z, axis=1)

In [5]:
# --- cell: load_problem_helpers ---
import numpy as np
import chaospy as cp

from sensitivity_examples_nonlinear import (
    analytic_sensitivity_coefficients,
)

In [6]:
# --- cell: define_problem_and_jpdf ---
# Define the (Z, W) non-additive linear demo problem and construct the joint distribution explicitly

Nrv = 4  # number of Z-variables (and also number of W-variables)

# Each row is [mu, sigma] for a Normal(mu, sigma)
zm = np.array([[0.0, i] for i in range(1, Nrv + 1)])

c = 0.5
wm = np.array([[i * c, i] for i in range(1, Nrv + 1)])

# --- Explicit construction of the joint PDF (independent marginals) ---
# Stack Z and W parameter definitions -> total parameter vector X = (Z1..ZNrv, W1..WNrv)
x_params = np.vstack([zm, wm])  # shape (2*Nrv, 2)

marginals = [cp.Normal(mu, sigma) for (mu, sigma) in x_params]
jpdf = cp.J(*marginals)  # independent joint distribution

# Analytic sensitivity values (reference)
Sa, Szw, Sta = analytic_sensitivity_coefficients(zm, wm)

print("Nrv =", Nrv, " -> total parameters =", len(jpdf))

Nrv = 4  -> total parameters = 8


In [7]:
# --- cell: mc_setup ---
# Monte Carlo sample size and basic dimensions

Ns_mc = 1000
P = len(jpdf)  # total number of stochastic inputs (e.g., Z and W stacked)

print("Ns_mc =", Ns_mc)
print("P     =", P)

Ns_mc = 1000
P     = 8


In [8]:
# --- cell: saltelli_matrices ---
# Step 1: create Saltelli sampling matrices A, B, and C_i

from monte_carlo import generate_sample_matrices_mc

A_s, B_s, C_s = generate_sample_matrices_mc(
    Ns_mc,
    P,
    jpdf,
    sample_method="R",
)

print("A_s:", np.shape(A_s))
print("B_s:", np.shape(B_s))
print("C_s:", np.shape(C_s))

A_s: (1000, 8)
B_s: (1000, 8)
C_s: (8, 1000, 8)


In [9]:
# --- cell: model_evaluation_saltelli ---
# Step 2: evaluate the model for A, B, and each C_i (glass box)

def eval_model(X):
    """
    Evaluate the non-additive linear demo model using the convention
    X = [Z1..Z_Nrv, W1..W_Nrv] (so total P = 2*Nrv).
    """
    Z = X[:, :Nrv]
    W = X[:, Nrv:]
    return linear_model(W, Z)


# Evaluate A and B
f_A = eval_model(A_s)
f_B = eval_model(B_s)

# Evaluate C_i. We support two common storage conventions for C_s:
#   (i)  C_s shape = (P, Ns, P)  -> C_s[i] is matrix C_i of shape (Ns, P)
#   (ii) C_s shape = (Ns, P, P)  -> C_s[:, i, :] is matrix C_i of shape (Ns, P)
C_shape = np.shape(C_s)

if len(C_shape) != 3:
    raise ValueError(f"Expected C_s to be 3D, got shape {C_shape}")

if C_shape[0] == P and C_shape[1] == Ns_mc and C_shape[2] == P:
    # convention (i): (P, Ns, P)
    f_C = np.zeros((P, Ns_mc))
    for i in range(P):
        f_C[i, :] = eval_model(C_s[i])
elif C_shape[0] == Ns_mc and C_shape[1] == P and C_shape[2] == P:
    # convention (ii): (Ns, P, P)
    f_C = np.zeros((P, Ns_mc))
    for i in range(P):
        f_C[i, :] = eval_model(C_s[:, i, :])
else:
    raise ValueError(
        "Unrecognized C_s layout. "
        f"Got C_s shape {C_shape}, expected (P, Ns_mc, P) or (Ns_mc, P, P)."
    )

print("f_A:", np.shape(f_A))
print("f_B:", np.shape(f_B))
print("f_C:", np.shape(f_C))

f_A: (1000,)
f_B: (1000,)
f_C: (8, 1000)


In [10]:
# --- cell: sobol_estimators ---
# Step 3: compute first-order (S) and total-order (ST) indices from model evaluations

from monte_carlo import calculate_sensitivity_indices_mc

Smc, Stmc = calculate_sensitivity_indices_mc(f_A, f_B, f_C)

labels = (
    [f"Z_{i}" for i in range(1, Nrv+1)] +
    [f"W_{i}" for i in range(1, Nrv+1)]
)

pretty_print_sobol_mc(Smc, Stmc, labels)


pretty_print_sobol_mc(Smc, Stmc)

,S (MC),ST (MC)
Z_1,0.940,0.948
Z_2,0.892,0.949
Z_3,0.725,0.940
Z_4,0.235,0.830
W_1,0.942,0.950
W_2,0.920,0.972
W_3,0.747,0.984
W_4,0.428,1.000


,S (MC),ST (MC)
X1,0.940,0.948
X2,0.892,0.949
X3,0.725,0.940
X4,0.235,0.830
X5,0.942,0.950
X6,0.920,0.972
X7,0.747,0.984
X8,0.428,1.000


        # --- cell: imports ---
        # (optional additional imports specific to MC notebook)


        # --- cell: problem_definition ---
        # Define model, parameters, and joint distribution (jpdf)


        # --- cell: sampling_matrices ---
        # Generate A, B, (and C_i) matrices for Saltelli / Sobol estimation


        # --- cell: model_evaluation ---
        # Evaluate model on sample matrices


        # --- cell: sobol_estimators ---
        # Compute S and ST (first-order and total) from the evaluations


        # --- cell: postprocess_tables ---
        # Build a summary table (e.g. pandas DataFrame) and print/pretty display


        # --- cell: plots ---
        # Plot indices and (optionally) convergence diagnostics
